<a href="https://colab.research.google.com/github/armen1s/BOT/blob/main/TitanV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ======================================================
# 🏗 МОДУЛЬ 1: MASTER DATA ENGINE (v1.2 - PROD READY)
# ======================================================
import os
import time
import warnings
from datetime import datetime

# 0. УСТАНОВКА (Можно закомментировать, если уже установлено)
# print("🛠 Подготовка инструментов...")
os.system("pip install ccxt pandas numpy numba -q")

import ccxt
import pandas as pd
import numpy as np
from numba import jit

# 1. НАСТРОЙКИ
SYMBOL = 'BTC/USD'
TIMEFRAME = '1h'
START_DATE = '2025-01-01 00:00:00' # Лучше брать больше истории для TFT
FILENAME = 'titan_raw_history.csv'
FINAL_FILE = 'titan_final_data.csv'

exchange = ccxt.bitstamp({'enableRateLimit': True})

# 2. СИНХРОНИЗАЦИЯ (С ЗАЩИТОЙ ОТ ОБРЫВОВ И БИТЫХ ФАЙЛОВ)
def sync_data():
    print(f"🚀 Синхронизация {SYMBOL}...")

    # Проверка на существование и валидность файла
    need_header = True
    if os.path.exists(FILENAME):
        if os.path.getsize(FILENAME) > 0:
            need_header = False
        else:
            print("⚠️ Файл найден, но он пуст. Пересоздаем.")

    if need_header:
        since_ts = int(pd.Timestamp(START_DATE).timestamp() * 1000)
        with open(FILENAME, 'w') as f:
            f.write("timestamp,open,high,low,close,volume\n")
    else:
        try:
            df_check = pd.read_csv(FILENAME)
            if not df_check.empty:
                since_ts = int(df_check.iloc[-1]['timestamp']) + 1
            else:
                since_ts = int(pd.Timestamp(START_DATE).timestamp() * 1000)
        except:
            since_ts = int(pd.Timestamp(START_DATE).timestamp() * 1000)

    while True:
        try:
            batch = exchange.fetch_ohlcv(SYMBOL, TIMEFRAME, since=since_ts, limit=1000)
            if not batch: break

            with open(FILENAME, 'a') as f:
                for c in batch:
                    f.write(f"{c[0]},{c[1]},{c[2]},{c[3]},{c[4]},{c[5]}\n")

            last_ts = batch[-1][0]
            # Если данные свежее, чем "сейчас - 1 час", останавливаемся
            if last_ts >= exchange.milliseconds() - 3600000:
                print("   ✅ История актуальна.")
                break

            since_ts = last_ts + 1
            print(".", end="", flush=True)
            time.sleep(exchange.rateLimit / 1000) # Соблюдаем лимиты биржи

        except Exception as e:
            print(f"\n⚠️ Ошибка сети: {e}. Ждем 5 сек..."); time.sleep(5)

# 3. МАТЕМАТИКА (Numba JIT)
@jit(nopython=True)
def calculate_fdi(prices, window=48):
    n = len(prices)
    res = np.full(n, np.nan)
    for i in range(window, n):
        seg = prices[i-window:i]
        mx, mn = np.max(seg), np.min(seg)
        if mx == mn: res[i] = 1.0; continue
        norm = (seg - mn) / (mx - mn)
        length = np.sum(np.sqrt((1/(window-1))**2 + np.diff(norm)**2))
        res[i] = 1 + (np.log(length) + np.log(2)) / np.log(2 * (window - 1))
    return res

def apply_forensics(df):
    df_c = df.copy()
    # Волатильность за 24 часа
    vol = df_c['close'].pct_change().rolling(24).std()
    body_max = np.maximum(df_c['open'], df_c['close'])
    expected = df_c['close'] * vol

    # Ищем аномальные шпили вверх (High слишком далеко от тела свечи относительно волатильности)
    anomalies = (df_c['high'] - body_max) / expected.replace(0, np.nan) > 4.0

    if anomalies.sum() > 0:
        print(f"   🧹 Forensics: Исправлено {anomalies.sum()} сбоев цены.")
        # Срезаем шпиль до разумного максимума (3 сигмы)
        df_c.loc[anomalies, 'high'] = body_max[anomalies] + (3.0 * expected[anomalies])
    return df_c

# 4. КОНВЕЙЕР ОБРАБОТКИ
if __name__ == "__main__":
    sync_data()
    print("\n⚙️ Feature Engineering...")

    df = pd.read_csv(FILENAME)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df = df.set_index('timestamp')
    df = df[~df.index.duplicated(keep='first')]

    # ==========================================
    # 🚑 ЛЕЧЕНИЕ ВРЕМЕНИ (Time Grid Repair) - FIXED
    # ==========================================
    # 1. Создаем идеальную сетку времени
    full_idx = pd.date_range(start=df.index.min(), end=df.index.max(), freq='h')
    df = df.reindex(full_idx)

    # 2. [CRITICAL FIX] Логика заполнения пустот
    # Если биржи не было в сети, объем = 0 (а не копируем предыдущий!)
    df['volume'] = df['volume'].fillna(0)

    # 3. Цены (Open, High, Low, Close) протягиваем вперед (Forward Fill)
    # Если торгов нет, цена остается последней известной
    df = df.ffill()

    # Forensics (чистка шпилек)
    df = apply_forensics(df)

    # Логарифмы (теперь безопасно, т.к. log1p(0) = 0)
    df['Close_Log'] = np.log(df['close'])
    df['Volume_Log'] = np.log1p(df['volume'])

    # Физика (Фрактальная размерность)
    df['FDI'] = calculate_fdi(df['close'].values)

    # Yang-Zhang Volatility
    w = 24
    ho = np.log(df['high'] / df['open'])
    lo = np.log(df['low'] / df['open'])
    co = np.log(df['close'] / df['open'])
    oc = np.log(df['open'] / df['close'].shift(1))

    yz_var = oc.rolling(w).var() + 0.134*co.rolling(w).var() + 0.866*((ho*(ho-co)) + (lo*(lo-co))).rolling(w).mean()
    df['YZ_Vol'] = np.sqrt(yz_var).fillna(0)

    # Volume Price Trend
    df['VPT'] = (df['Volume_Log'] * df['Close_Log'].diff()).cumsum().fillna(0)

    # Структура для TFT (Temporal Fusion Transformer)
    df = df.sort_index()
    df['time_idx'] = range(len(df)) # Теперь это идеальная непрерывная последовательность
    df['group_id'] = 'BTC'          # Статическая группа для TFT
    df['hour'] = df.index.hour.astype(str).astype("category")
    df['day'] = df.index.dayofweek.astype(str).astype("category")

    # Удаляем NaN, которые неизбежно возникли в начале из-за скользящих окон (FDI, YZ)
    df.dropna(inplace=True)

    # Сбрасываем индекс, чтобы timestamp стал колонкой в CSV
    df.reset_index(inplace=True)
    df = df.rename(columns={'index': 'timestamp'})

    df.to_csv(FINAL_FILE, index=False)

    print("\n" + "="*50)
    print(f"✅ DATA ENGINE ЗАВЕРШЕН.")
    print(f"   Баров: {len(df)}")
    print(f"   Пропусков времени: 0 (Grid Repaired & Volume Fixed)")
    print(f"📂 Файл готов: {FINAL_FILE}")
    print("="*50)

🚀 Синхронизация BTC/USD...
.........   ✅ История актуальна.

⚙️ Feature Engineering...
   🧹 Forensics: Исправлено 1 сбоев цены.

✅ DATA ENGINE ЗАВЕРШЕН.
   Баров: 9325
   Пропусков времени: 0 (Grid Repaired & Volume Fixed)
📂 Файл готов: titan_final_data.csv


In [ ]:
!pip install lightning pytorch_forecasting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.0/846.0 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.8/399.8 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.8/159.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 14.4 MB/s eta 0:00:00


In [3]:
# ======================================================
# 🏆 TITAN AI: GOLDEN CONFIG (RESTART SESSION FIRST!)
# ======================================================
import os
import sys

# 1. СТАБИЛИЗАЦИЯ
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
import subprocess
import warnings
import shutil
import numpy as np
import pandas as pd
import torch
from lightning.pytorch.callbacks import Callback

# НАСТРОЙКИ (Те самые, рабочие)
MODE = "NORMAL"

# Детерминизм
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
try:
    torch.use_deterministic_algorithms(True, warn_only=True)
except: pass

# 2. АВТО-УСТАНОВКА
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])

try:
    import lightning.pytorch as pl
    from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
    from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
    from pytorch_forecasting.data import GroupNormalizer
    from pytorch_forecasting.metrics import QuantileLoss
except ImportError:
    install("lightning")
    install("pytorch_forecasting")
    import lightning.pytorch as pl
    from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
    from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
    from pytorch_forecasting.data import GroupNormalizer
    from pytorch_forecasting.metrics import QuantileLoss

# 3. ДАННЫЕ
pl.seed_everything(42)
warnings.filterwarnings("ignore")

df = pd.read_csv('titan_final_data.csv')
df['group_id'] = df['group_id'].astype(str)
df['hour'] = df['hour'].astype(str)
df['day'] = df['day'].astype(str)
df['time_idx'] = df['time_idx'].astype(int)

# Разделение
BLIND_TEST_HOURS = 720
last_idx = df["time_idx"].max()
cutoff_idx = last_idx - BLIND_TEST_HOURS

ENCODER_LENGTH = 48
PREDICTION_LENGTH = 12

training_dataset = TimeSeriesDataSet(
    df[lambda x: x.time_idx <= cutoff_idx],
    time_idx="time_idx",
    target="Close_Log",
    group_ids=["group_id"],
    min_encoder_length=ENCODER_LENGTH, max_encoder_length=ENCODER_LENGTH,
    min_prediction_length=PREDICTION_LENGTH, max_prediction_length=PREDICTION_LENGTH,
    static_categoricals=["group_id"],
    time_varying_known_categoricals=["hour", "day"],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_reals=["Close_Log", "Volume_Log", "YZ_Vol", "FDI", "VPT"],
    target_normalizer=GroupNormalizer(groups=["group_id"], transformation="softplus"),
    add_relative_time_idx=True, add_target_scales=True, add_encoder_length=True,
)

validation = TimeSeriesDataSet.from_dataset(
    training_dataset, df, predict=False, stop_randomization=True, min_prediction_idx=cutoff_idx + 1
)

batch_size = 64
train_dataloader = training_dataset.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

# 4. МОДЕЛЬ (ВОЗВРАТ К РАБОЧИМ ПАРАМЕТРАМ v2.0)
print(f"\n🧠 ЗАПУСК ОБУЧЕНИЯ (Golden Config)...")

tft = TemporalFusionTransformer.from_dataset(
    training_dataset,
    # ВАЖНО: 0.02, а не 0.1 !!!
    learning_rate=0.02,

    # ВАЖНО: Возвращаем "мозги", которые давали прибыль
    hidden_size=16,
    hidden_continuous_size=8,
    dropout=0.15,

    attention_head_size=1,
    output_size=7,
    loss=QuantileLoss(),
)

checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode="min", save_top_k=1, filename="titan_best")
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=5, verbose=True, mode="min")

trainer = pl.Trainer(
    max_epochs=15,
    accelerator="auto",
    devices=1,
    callbacks=[early_stop_callback, checkpoint_callback],
    enable_model_summary=True,
    gradient_clip_val=0.1,
)

trainer.fit(tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

# 5. СИМУЛЯЦИЯ (DIAMOND HANDS v3.0)
print("\n" + "="*60)
print("💰 СИМУЛЯЦИЯ...")
print("="*60)

best_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_path)

raw_predictions = best_tft.predict(val_dataloader, mode="raw", return_x=True, return_index=True)
preds = raw_predictions.output.prediction
x = raw_predictions.x
index_df = raw_predictions.index # Исправляет ошибку индекса

# Параметры стратегии
STRICT_RISK = 0.035       # 3.5%
MIN_ROI_ENTRY = 0.0055    # 0.55%
COMMISSION = 0.002

balance = 1000.0
trades = 0
wins = 0
busy_until_idx = -1

# Синхронизация для цикла
n_samples = min(preds.shape[0], len(index_df))
if isinstance(index_df, pd.DataFrame): index_df = index_df.reset_index(drop=True)

for i in range(n_samples):
    prediction_start_idx = int(index_df.iloc[i]['time_idx'])
    decision_idx = prediction_start_idx - 1

    if decision_idx < busy_until_idx: continue

    log_forecast = preds[i, :, 3].cpu().detach().numpy()[-1]
    log_optimist = preds[i, :, 5].cpu().detach().numpy()[-1]
    log_pessimist = preds[i, :, 1].cpu().detach().numpy()[-1]

    log_curr = x["encoder_target"][i][-1].cpu().detach().numpy()
    curr_p = np.exp(log_curr)
    targ_p = np.exp(log_forecast)

    roi = (targ_p - curr_p) / curr_p
    risk_spread = (np.exp(log_optimist) - np.exp(log_pessimist)) / targ_p

    signal = "NONE"
    if risk_spread <= STRICT_RISK:
        if roi > MIN_ROI_ENTRY: signal = "LONG"
        elif roi < -MIN_ROI_ENTRY: signal = "SHORT"

    if signal != "NONE":
        # Упрощенный выход (через 12 часов) для проверки
        log_exit = x["decoder_target"][i][-1].cpu().detach().numpy()
        exit_p = np.exp(log_exit)

        pnl = (exit_p - curr_p) / curr_p if signal == "LONG" else (curr_p - exit_p) / curr_p
        net = pnl - COMMISSION
        balance *= (1 + net)

        if net > 0: wins += 1
        trades += 1
        busy_until_idx = decision_idx + 12

color = "\033[92m" if balance > 1000 else "\033[91m"
print(f"Итог (v2.0): {trades} сделок | WR: {wins/trades*100 if trades else 0:.1f}% | {color}${balance:.2f}\033[0m")

Epoch 8/14 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133/133 0:00:17 • 0:00:00 7.77it/s v_num: 0.000 train_loss_step:     
                                                                                 0.006 val_loss: 0.005             
                                                                                 train_loss_epoch: 0.006           

INFO: Monitored metric val_loss did not improve in the last 5 records. Best score: 0.005. Signaling Trainer to stop.
INFO:lightning.pytorch.callbacks.early_stopping:Monitored metric val_loss did not improve in the last 5 records. Best score: 0.005. Signaling Trainer to stop.



💰 СИМУЛЯЦИЯ...


INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Итог (v2.0): 29 сделок | WR: 58.6% | $1066.78


In [10]:
# --- 8. СИМУЛЯЦИЯ (ИСПРАВЛЕННАЯ v2.5) ---
print("\n" + "="*60)
print("💰 СИМУЛЯЦИЯ ТОРГОВЛИ (ЧЕСТНЫЙ РЕЖИМ)")
print("="*60)

best_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_path)

# Получаем прогнозы и индексы
raw_predictions = best_tft.predict(val_dataloader, mode="raw", return_x=True, return_index=True)
preds = raw_predictions.output.prediction
x = raw_predictions.x
index_df = raw_predictions.index  # <--- Вот наш надежный источник времени!

idx_to_date = df.set_index("time_idx")["timestamp"].to_dict()

def run_simulation(name, profit_target, risk_limit):
    balance = 1000.0
    trades = []
    wins = 0
    busy_until_idx = -1

    n_samples = preds.shape[0]

    for i in range(n_samples):
        # В index_df лежит time_idx ПЕРВОГО предсказываемого часа (decoder start)
        # Значит, решение мы принимаем на 1 час раньше (конец encoder)
        prediction_start_idx = int(index_df.iloc[i]['time_idx'])
        decision_idx = prediction_start_idx - 1

        # Фильтр занятости капитала
        if decision_idx < busy_until_idx: continue

        # Прогнозы (Квантили: 1=10%, 3=50%, 5=90%)
        log_forecast = preds[i, :, 3].cpu().detach().numpy()[-1]
        log_optimist = preds[i, :, 5].cpu().detach().numpy()[-1]
        log_pessimist = preds[i, :, 1].cpu().detach().numpy()[-1]

        # Цена входа (последняя известная цена из энкодера)
        log_entry = x["encoder_target"][i][-1].cpu().detach().numpy()
        entry_price = np.exp(log_entry)
        target_price = np.exp(log_forecast)

        roi = (target_price - entry_price) / entry_price
        risk_spread = (np.exp(log_optimist) - np.exp(log_pessimist)) / target_price

        # Логика сигнала
        signal = "NONE"
        if risk_spread <= risk_limit:
            if roi > profit_target: signal = "LONG"
            elif roi < -profit_target: signal = "SHORT"

        # Исполнение
        if signal != "NONE":
            # Цена выхода (берем из decoder_target - это будущее)
            log_exit = x["decoder_target"][i][-1].cpu().detach().numpy()
            exit_price = np.exp(log_exit)

            if signal == "LONG": pnl = (exit_price - entry_price) / entry_price
            else: pnl = (entry_price - exit_price) / entry_price

            net_pnl = pnl - 0.002 # Комиссия 0.2%
            balance *= (1 + net_pnl)

            if net_pnl > 0: wins += 1

            # Блокируем торговлю на время удержания позиции (12 часов)
            busy_until_idx = decision_idx + PREDICTION_LENGTH

            ts_val = idx_to_date.get(decision_idx, f"idx_{decision_idx}")
            trades.append({'Time': ts_val, 'Type': signal, 'PnL': round(net_pnl*100, 2), 'Bal': int(balance)})

    count = len(trades)
    winrate = (wins/count*100) if count > 0 else 0
    color = "\033[92m" if balance > 1000 else "\033[91m"
    print(f"📊 {name:10} | Сделок: {count:3} | WR: {winrate:.1f}% | Итог: {color}${balance:.2f}\033[0m")

    # Показать примеры сделок, если они есть
    if count > 0:
        print(f"   Последние 2 сделки: {trades[-2:]}")

run_simulation("INVESTOR", profit_target=0.015, risk_limit=0.06)
run_simulation("TRADER",   profit_target=0.008, risk_limit=0.04)
run_simulation("SCALPER",  profit_target=0.004, risk_limit=0.02)


💰 СИМУЛЯЦИЯ ТОРГОВЛИ (ЧЕСТНЫЙ РЕЖИМ)


INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


📊 INVESTOR   | Сделок:   7 | WR: 42.9% | Итог: $1032.55
   Последние 2 сделки: [{'Time': '2026-01-19 08:00:00', 'Type': 'SHORT', 'PnL': np.float32(-0.07), 'Bal': 1007}, {'Time': '2026-01-21 16:00:00', 'Type': 'LONG', 'PnL': np.float32(2.51), 'Bal': 1032}]
📊 TRADER     | Сделок:  19 | WR: 63.2% | Итог: $1040.77
   Последние 2 сделки: [{'Time': '2026-01-21 16:00:00', 'Type': 'LONG', 'PnL': np.float32(2.51), 'Bal': 1032}, {'Time': '2026-01-22 04:00:00', 'Type': 'SHORT', 'PnL': np.float32(0.84), 'Bal': 1040}]
📊 SCALPER    | Сделок:   0 | WR: 0.0% | Итог: $1000.00


In [11]:

# ======================================================
# 🚑 DIAGNOSTIC REBOOT: ПРИНУДИТЕЛЬНЫЙ БЭКТЕСТ
# ======================================================
import torch
import pandas as pd
import numpy as np
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer

print("🔄 ПЕРЕКЛЮЧЕНИЕ В РЕЖИМ ИСТОРИИ (BACKTEST MODE)...")

# 1. Загружаем данные заново (последние 35 дней для запаса)
df_full = pd.read_csv('titan_final_data.csv')
# Исправляем типы
df_full['group_id'] = df_full['group_id'].astype(str)
df_full['hour'] = df_full['hour'].astype(str)
df_full['day'] = df_full['day'].astype(str)
df_full['time_idx'] = df_full['time_idx'].astype(int)

# Берем хвост (30 дней + запас на энкодер 48ч)
backtest_data = df_full.tail(800).copy()

# 2. Создаем датасет с флагом predict=False (ЭТО ГЛАВНОЕ!)
# predict=False заставляет модель идти по истории, а не прыгать в будущее
backtest_ds = TimeSeriesDataSet.from_dataset(
    training_dataset,
    backtest_data,
    predict=False,
    stop_randomization=True
)
backtest_dl = backtest_ds.to_dataloader(train=False, batch_size=128, num_workers=0)

# 3. Загружаем модель
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

# 4. Прогоняем историю
print(f"🧠 Сканирование {len(backtest_data)} часов истории...")
raw_predictions = best_tft.predict(backtest_dl, mode="raw", return_x=True, return_index=True)

preds = raw_predictions.output.prediction
x = raw_predictions.x

print(f"✅ УСПЕХ! Получено прогнозов: {preds.shape[0]}")
print("-" * 75)

# ======================================================
# 🩻 ЗАПУСК РЕНТГЕНА (ТЕПЕРЬ ДАННЫЕ ЕСТЬ)
# ======================================================
print("🔎 АНАЛИЗ ПРИЧИН МОЛЧАНИЯ БОТА...")

total_signals = 0
high_risk_count = 0
missed_profits = 0
risk_log = []

print(f"{'BAR':<6} | {'TYPE':<5} | {'ROI %':<8} | {'RISK %':<10} | {'REAL PNL':<10} | {'RESULT'}")
print("-" * 75)

# Проходим по всем доступным прогнозам
for i in range(preds.shape[0]):
    # Данные
    log_forecast = preds[i, :, 3].cpu().detach().numpy()[-1]
    log_optimist = preds[i, :, 5].cpu().detach().numpy()[-1] # p90
    log_pessimist = preds[i, :, 1].cpu().detach().numpy()[-1] # p10

    # Вход (Encoder Target - последняя точка)
    log_curr = x["encoder_target"][i][-1].cpu().detach().numpy()
    curr = np.exp(log_curr)
    targ = np.exp(log_forecast)

    # Метрики
    roi = (targ - curr) / curr
    # Spread = (Optimist - Pessimist) / Target
    risk_spread = (np.exp(log_optimist) - np.exp(log_pessimist)) / targ

    risk_log.append(risk_spread)

    # Фильтр интереса (смотрим всё, где профит > 0.3%)
    if abs(roi) > 0.003:
        total_signals += 1
        signal = "LONG" if roi > 0 else "SHORT"

        # Реальность (Decoder Target - последняя точка)
        log_exit = x["decoder_target"][i][-1].cpu().detach().numpy()
        exit_p = np.exp(log_exit)

        if exit_p < 1000: continue

        pnl = (exit_p - curr) / curr if signal == "LONG" else (curr - exit_p) / curr
        is_win = pnl > 0.002 # С учетом комиссии

        # Анализ страха (> 8% риск)
        risk_status = ""
        if risk_spread > 0.08:
            high_risk_count += 1
            risk_color = "\033[91m" # Красный
            if is_win:
                missed_profits += 1
                risk_status = "(УПУЩЕНО)"
        else:
            risk_color = "\033[92m" # Зеленый

        # Показываем строки
        if i % 20 == 0 or (is_win and risk_spread > 0.08):
             print(f"{i:<6} | {signal:<5} | {roi*100:>6.2f}%  | {risk_color}{risk_spread*100:>6.2f}%\033[0m   | {pnl*100:>+6.2f}%     | {'✅' if is_win else '❌'} {risk_status}")

# ИТОГ
if len(risk_log) > 0:
    avg_risk = np.mean(risk_log)
    min_risk = np.min(risk_log)

    print("-" * 75)
    print(f"📊 СТАТИСТИКА СТРАХА:")
    print(f"   Средний риск: {avg_risk*100:.2f}%")
    print(f"   Минимальный риск: {min_risk*100:.2f}% <--- ЭТО КЛЮЧЕВАЯ ЦИФРА")
    print(f"   Всего сигналов: {total_signals}")
    print(f"   Отменено из-за страха (>8%): {high_risk_count}")
    print(f"   Упущенные деньги (прибыльные, но страшные): {missed_profits}")

    print("\n💡 РЕКОМЕНДАЦИЯ:")
    if min_risk > 0.08:
        target_risk = min_risk + 0.015
        print(f"   Рынок был очень опасным. Чтобы торговать, установите RISK_LIMIT = {target_risk:.2f}")
    else:
        print(f"   Риски были в норме. Проблема в фильтре ROI. Попробуйте снизить PROFIT_TARGET до 0.003")
else:
    print("⚠️ Странно, массив рисков пуст. Проверьте данные.")

🔄 ПЕРЕКЛЮЧЕНИЕ В РЕЖИМ ИСТОРИИ (BACKTEST MODE)...


INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


🧠 Сканирование 800 часов истории...
✅ УСПЕХ! Получено прогнозов: 741
---------------------------------------------------------------------------
🔎 АНАЛИЗ ПРИЧИН МОЛЧАНИЯ БОТА...
BAR    | TYPE  | ROI %    | RISK %     | REAL PNL   | RESULT
---------------------------------------------------------------------------
0      | SHORT |  -0.67%  |   2.69%   |  +2.38%     | ✅ 
20     | LONG  |   1.03%  |   2.48%   |  +0.10%     | ❌ 
40     | LONG  |   0.76%  |   2.58%   |  +0.27%     | ✅ 
60     | LONG  |   0.35%  |   2.93%   |  +2.63%     | ✅ 
80     | SHORT |  -0.37%  |   3.18%   |  +0.15%     | ❌ 
140    | LONG  |   0.39%  |   2.86%   |  +0.19%     | ❌ 
180    | SHORT |  -0.54%  |   2.81%   |  +0.22%     | ✅ 
200    | SHORT |  -0.36%  |   3.14%   |  -1.47%     | ❌ 
220    | SHORT |  -1.19%  |   3.24%   |  +0.08%     | ❌ 
240    | SHORT |  -0.99%  |   4.40%   |  -1.22%     | ❌ 
260    | SHORT |  -1.39%  |   4.39%   |  -0.03%     | ❌ 
280    | SHORT |  -0.53%  |   4.25%   |  -0.22%     | ❌ 
3

In [12]:
# ======================================================
# 💎 ТЕСТ "DIAMOND HANDS": STABLE VERSION
# ======================================================
import numpy as np
import pandas as pd

# 1. ПРОВЕРКА ДАННЫХ (Fix IndexError)
# Убедимся, что index_df существует и совпадает по длине
try:
    # Принудительно сбрасываем индекс, чтобы iloc[i] работал корректно
    if isinstance(index_df, pd.DataFrame):
        index_df = index_df.reset_index(drop=True)

    # Безопасная длина: берем минимум между прогнозами и датами
    n_preds = preds.shape[0]
    n_idx = len(index_df)
    n_samples = min(n_preds, n_idx)

    print(f"✅ Данные синхронизированы. Доступно семплов: {n_samples}")

except NameError:
    print("❌ ОШИБКА: Переменная 'index_df' или 'preds' не найдена.")
    print("   Запустите блок 'СИМУЛЯЦИЯ (ЧЕСТНЫЙ РЕЖИМ)' выше, чтобы они создались.")
    n_samples = 0

# 2. НАСТРОЙКИ
STRICT_RISK = 0.033 # 3.5% (Это 0.035, а не 0.28!)

def run_strict_test(profit_target):
    if n_samples == 0: return

    balance = 1000.0
    trades = 0
    wins = 0
    busy_until_idx = -1

    for i in range(n_samples):
        # 0. Время
        # Теперь это безопасно, так как мы выровняли n_samples
        prediction_start_idx = int(index_df.iloc[i]['time_idx'])
        decision_idx = prediction_start_idx - 1

        # ФИЛЬТР: Занятость капитала
        if decision_idx < busy_until_idx:
            continue

        # 1. Прогноз
        log_forecast = preds[i, :, 3].cpu().detach().numpy()[-1]
        log_optimist = preds[i, :, 5].cpu().detach().numpy()[-1]
        log_pessimist = preds[i, :, 1].cpu().detach().numpy()[-1]

        log_curr = x["encoder_target"][i][-1].cpu().detach().numpy()
        curr = np.exp(log_curr)
        targ = np.exp(log_forecast)

        # 2. Метрики
        roi = (targ - curr) / curr
        risk_spread = (np.exp(log_optimist) - np.exp(log_pessimist)) / targ

        # 3. Сигнал (Строгий риск)
        signal = "NONE"
        if risk_spread <= STRICT_RISK:
            if roi > profit_target: signal = "LONG"
            elif roi < -profit_target: signal = "SHORT"

        # 4. Исполнение
        if signal != "NONE":
            log_exit = x["decoder_target"][i][-1].cpu().detach().numpy()
            exit_p = np.exp(log_exit)

            # Фильтр битых цен
            if exit_p < 0.0001: continue

            pnl = (exit_p - curr) / curr if signal == "LONG" else (curr - exit_p) / curr
            net = pnl - 0.002

            balance *= (1 + net)
            trades += 1
            if net > 0: wins += 1

            # Блокировка (12 часов удержания)
            busy_until_idx = decision_idx + 12

    # Вывод
    color = "\033[92m" if balance > 1000 else "\033[91m"
    wr = (wins / trades * 100) if trades > 0 else 0.0
    print(f"Target > {profit_target*100:.1f}% | Сделок: {trades:3} | WR: {wr:5.1f}% | Итог: {color}${balance:.2f}\033[0m")

print("-" * 60)
print(f"💎 DIAMOND HANDS TEST (Risk limit: {STRICT_RISK*100}%)")
print("-" * 60)

# ЗАПУСК
run_strict_test(0.009) # 0.9% тейк
run_strict_test(0.005) # 0.5% тейк
run_strict_test(0.003) # 0.3% тейк

✅ Данные синхронизированы. Доступно семплов: 709
------------------------------------------------------------
💎 DIAMOND HANDS TEST (Risk limit: 3.3000000000000003%)
------------------------------------------------------------
Target > 0.9% | Сделок:  14 | WR:  57.1% | Итог: $1059.19
Target > 0.5% | Сделок:  28 | WR:  57.1% | Итог: $1092.70
Target > 0.3% | Сделок:  34 | WR:  52.9% | Итог: $1094.66


In [7]:

# ======================================================
# 💎 DIAMOND HANDS 3.0: ЧЕСТНЫЙ ТРЕЙЛИНГ (REALISTIC EXECUTION)
# ======================================================
import numpy as np

# НАСТРОЙКИ
STRICT_RISK = 0.035       # Риск 3.5%
MIN_ROI_ENTRY = 0.0055    # Вход > 0.55%
COMMISSION = 0.002        # Комиссия 0.2%

print(f"💎 ЗАПУСК TRAILING STOP (С УЧЕТОМ ПРОСКАЛЬЗЫВАНИЯ)")
print(f"🔒 Risk Limit: {STRICT_RISK*100}% | Min ROI: {MIN_ROI_ENTRY*100}%")
print("-" * 75)

def get_volatility(encoder_seq):
    prices = np.exp(encoder_seq[-12:].cpu().detach().numpy())
    return np.std(prices) / np.mean(prices)

def run_trailing_strategy_verified(name):
    balance = 1000
    trades = 0
    wins = 0

    limit = preds.shape[0] - 12

    for i in range(limit):
        # --- 1. ВХОД (TITAN SIGNALS) ---
        log_forecast = preds[i, :, 3].cpu().detach().numpy()[-1]
        log_optimist = preds[i, :, 5].cpu().detach().numpy()[-1]
        log_pessimist = preds[i, :, 1].cpu().detach().numpy()[-1]

        log_curr = x["encoder_target"][i][-1].cpu().detach().numpy()
        curr_p = np.exp(log_curr)
        targ_p = np.exp(log_forecast)

        roi = (targ_p - curr_p) / curr_p
        risk_spread = (np.exp(log_optimist) - np.exp(log_pessimist)) / targ_p

        signal = "NONE"
        if risk_spread <= STRICT_RISK:
            if roi > MIN_ROI_ENTRY: signal = "LONG"
            elif roi < -MIN_ROI_ENTRY: signal = "SHORT"

        # --- 2. ВЫХОД (TRAILING STOP) ---
        if signal != "NONE":
            log_future = x["decoder_target"][i].cpu().detach().numpy()
            future_prices = np.exp(log_future)

            # ДИСТАНЦИЯ ТРЕЙЛИНГА
            if "СКАЛЬПЕР" in name:
                TRAIL_PCT = 0.0055  # 0.55%
            elif "ИМПУЛЬС" in name:
                TRAIL_PCT = 0.0075  # 0.75%
            elif "ATR" in name:
                vol = get_volatility(x["encoder_target"][i])
                TRAIL_PCT = max(vol, 0.005) # Не менее 0.5%

            exit_p = None

            if signal == "LONG":
                # Стартовый стоп
                current_sl = curr_p * (1 - TRAIL_PCT)
                highest_price = curr_p

                for price in future_prices:
                    # Подтяжка стопа (ТОЛЬКО ВВЕРХ)
                    if price > highest_price:
                        highest_price = price
                        new_sl = highest_price * (1 - TRAIL_PCT)
                        if new_sl > current_sl:
                            current_sl = new_sl

                    # Проверка выхода
                    if price <= current_sl:
                        # !!! ИСПРАВЛЕНИЕ: Выходим по рынку (price), а не по стопу !!!
                        # Это учитывает проскальзывание против нас
                        exit_p = price
                        break

            elif signal == "SHORT":
                # Стартовый стоп
                current_sl = curr_p * (1 + TRAIL_PCT)
                lowest_price = curr_p

                for price in future_prices:
                    # Подтяжка стопа (ТОЛЬКО ВНИЗ)
                    if price < lowest_price:
                        lowest_price = price
                        new_sl = lowest_price * (1 + TRAIL_PCT)
                        if new_sl < current_sl:
                            current_sl = new_sl

                    # Проверка выхода
                    if price >= current_sl:
                        # !!! ИСПРАВЛЕНИЕ !!!
                        exit_p = price
                        break

            # Если стоп не выбило за 12 часов - выход по последней цене
            if exit_p is None:
                exit_p = future_prices[-1]

            # --- 3. РАСЧЕТ PNL ---
            if signal == "LONG":
                raw_pnl = (exit_p - curr_p) / curr_p
            else:
                raw_pnl = (curr_p - exit_p) / curr_p

            net_pnl = raw_pnl - COMMISSION
            balance *= (1 + net_pnl)

            trades += 1
            if net_pnl > 0: wins += 1

    # ВЫВОД
    color = "\033[92m" if balance > 1000 else "\033[91m"
    res_pct = (balance - 1000)/10

    print(f"📌 {name:<15} | Deals: {trades:3} | WR: {wins/trades*100 if trades>0 else 0:.1f}% | {color}${balance:.2f} ({res_pct:+.1f}%)\033[0m")

# ЗАПУСК
print(f"{'STRATEGY':<17} | {'STATS':<20} | {'RESULT'}")
print("-" * 75)

run_trailing_strategy_verified("А. СКАЛЬПЕР")
run_trailing_strategy_verified("Б. ИМПУЛЬС")
run_trailing_strategy_verified("В. ATR (SMART)")

print("-" * 75)

💎 ЗАПУСК TRAILING STOP (С УЧЕТОМ ПРОСКАЛЬЗЫВАНИЯ)
🔒 Risk Limit: 3.5000000000000004% | Min ROI: 0.5499999999999999%
---------------------------------------------------------------------------
STRATEGY          | STATS                | RESULT
---------------------------------------------------------------------------
📌 А. СКАЛЬПЕР     | Deals: 181 | WR: 46.4% | $912.96 (-8.7%)
📌 Б. ИМПУЛЬС      | Deals: 181 | WR: 44.8% | $1019.71 (+2.0%)
📌 В. ATR (SMART)  | Deals: 181 | WR: 45.3% | $949.60 (-5.0%)
---------------------------------------------------------------------------


In [8]:
# ======================================================
# 💎 DIAMOND HANDS 3.0: ВЕРИФИЦИРОВАННЫЙ КОД
# ======================================================
import numpy as np

# НАСТРОЙКИ (КОНСЕРВАТИВНЫЕ)
STRICT_RISK = 0.035       # Риск 3.5%
MIN_ROI_ENTRY = 0.0055    # Вход > 0.55%
COMMISSION = 0.002        # 0.2% (Включает проскальзывание)

print(f"💎 ЗАПУСК ВЕРИФИЦИРОВАННОЙ СИМУЛЯЦИИ")
print(f"🔒 Risk Limit: {STRICT_RISK*100}% | Min ROI: {MIN_ROI_ENTRY*100}%")
print("-" * 75)

def get_volatility(encoder_seq):
    # Считаем волатильность по 12 прошлым точкам
    prices = np.exp(encoder_seq[-12:].cpu().detach().numpy())
    return np.std(prices) / np.mean(prices)

def run_strategy_final_check(name):
    balance = 1000
    trades = 0
    wins = 0

    # Защита от выхода за границы массива
    limit = preds.shape[0] - 12

    for i in range(limit):
        # ------------------------------------------------
        # 1. ЛОГИКА ВХОДА (ENTRY)
        # ------------------------------------------------
        log_forecast = preds[i, :, 3].cpu().detach().numpy()[-1] # p50
        log_optimist = preds[i, :, 5].cpu().detach().numpy()[-1] # p90
        log_pessimist = preds[i, :, 1].cpu().detach().numpy()[-1] # p10

        # Берем ПОСЛЕДНЮЮ известную цену перед прогнозом
        log_curr = x["encoder_target"][i][-1].cpu().detach().numpy()

        curr_p = np.exp(log_curr)
        targ_p = np.exp(log_forecast)

        roi = (targ_p - curr_p) / curr_p
        risk_spread = (np.exp(log_optimist) - np.exp(log_pessimist)) / targ_p

        signal = "NONE"

        # Фильтр: Низкий риск + Достаточный потенциал
        if risk_spread <= STRICT_RISK:
            if roi > MIN_ROI_ENTRY: signal = "LONG"
            elif roi < -MIN_ROI_ENTRY: signal = "SHORT"

        # ------------------------------------------------
        # 2. ЛОГИКА ВЫХОДА (EXIT - TRAILING STOP)
        # ------------------------------------------------
        if signal != "NONE":
            # Загружаем будущее (12 часов)
            log_future = x["decoder_target"][i].cpu().detach().numpy()
            future_prices = np.exp(log_future)

            # Определяем дистанцию трейлинга
            if "СКАЛЬПЕР" in name:
                TRAIL_PCT = 0.0055
            elif "ИМПУЛЬС" in name:
                TRAIL_PCT = 0.0075
            elif "ATR" in name:
                vol = get_volatility(x["encoder_target"][i])
                TRAIL_PCT = max(vol, 0.005) # Минимум 0.5%

            exit_p = None

            # --- СИМУЛЯЦИЯ LONG ---
            if signal == "LONG":
                current_sl = curr_p * (1 - TRAIL_PCT) # Начальный стоп
                highest_price = curr_p

                for price in future_prices:
                    # Трейлинг: Стоп ползет ВВЕРХ
                    if price > highest_price:
                        highest_price = price
                        new_sl = highest_price * (1 - TRAIL_PCT)
                        if new_sl > current_sl:
                            current_sl = new_sl

                    # Проверка удара об стоп
                    if price <= current_sl:
                        exit_p = price # Выход по цене закрытия свечи (Честный вариант)
                        break

            # --- СИМУЛЯЦИЯ SHORT ---
            elif signal == "SHORT":
                current_sl = curr_p * (1 + TRAIL_PCT) # Начальный стоп
                lowest_price = curr_p

                for price in future_prices:
                    # Трейлинг: Стоп ползет ВНИЗ
                    if price < lowest_price:
                        lowest_price = price
                        new_sl = lowest_price * (1 + TRAIL_PCT)
                        if new_sl < current_sl:
                            current_sl = new_sl

                    # Проверка удара об стоп
                    if price >= current_sl:
                        exit_p = price # Выход по цене закрытия
                        break

            # Если стоп не выбило за 12 часов -> выход по последней цене
            if exit_p is None:
                exit_p = future_prices[-1]

            # ------------------------------------------------
            # 3. РАСЧЕТ PNL
            # ------------------------------------------------
            if signal == "LONG":
                raw_pnl = (exit_p - curr_p) / curr_p
            else:
                raw_pnl = (curr_p - exit_p) / curr_p

            net_pnl = raw_pnl - COMMISSION
            balance *= (1 + net_pnl)

            trades += 1
            if net_pnl > 0: wins += 1

    # ВЫВОД
    color = "\033[92m" if balance > 1000 else "\033[91m"
    res_pct = (balance - 1000)/10

    print(f"📌 {name:<15} | Deals: {trades:3} | WR: {wins/trades*100 if trades>0 else 0:.1f}% | {color}${balance:.2f} ({res_pct:+.1f}%)\033[0m")

# ЗАПУСК
print(f"{'STRATEGY':<17} | {'STATS':<20} | {'RESULT'}")
print("-" * 75)

run_strategy_final_check("А. СКАЛЬПЕР")
run_strategy_final_check("Б. ИМПУЛЬС")
run_strategy_final_check("В. ATR (SMART)")

print("-" * 75)

💎 ЗАПУСК ВЕРИФИЦИРОВАННОЙ СИМУЛЯЦИИ
🔒 Risk Limit: 3.5000000000000004% | Min ROI: 0.5499999999999999%
---------------------------------------------------------------------------
STRATEGY          | STATS                | RESULT
---------------------------------------------------------------------------
📌 А. СКАЛЬПЕР     | Deals: 181 | WR: 46.4% | $912.96 (-8.7%)
📌 Б. ИМПУЛЬС      | Deals: 181 | WR: 44.8% | $1019.71 (+2.0%)
📌 В. ATR (SMART)  | Deals: 181 | WR: 45.3% | $949.60 (-5.0%)
---------------------------------------------------------------------------


In [9]:

# ======================================================
# 💎 DIAMOND HANDS 3.0: ЧЕСТНЫЙ ТРЕЙЛИНГ (REALISTIC EXECUTION)
# ======================================================
import numpy as np
import pandas as pd

# 1. СИНХРОНИЗАЦИЯ ДАННЫХ (Защита от ошибок индекса)
try:
    if isinstance(index_df, pd.DataFrame):
        index_df = index_df.reset_index(drop=True)
    n_preds = preds.shape[0]
    n_idx = len(index_df)
    n_samples = min(n_preds, n_idx)
    print(f"✅ Данные синхронизированы. Доступно семплов: {n_samples}")
except NameError:
    print("❌ ОШИБКА: Запустите предыдущие блоки (где создаются preds и index_df)!")
    n_samples = 0

# 2. НАСТРОЙКИ
STRICT_RISK = 0.028       # Риск 3.5%
MIN_ROI_ENTRY = 0.0055    # Вход > 0.55%
COMMISSION = 0.002        # Комиссия 0.2%

print(f"💎 ЗАПУСК TRAILING STOP (С УЧЕТОМ ПРОСКАЛЬЗЫВАНИЯ)")
print(f"🔒 Risk Limit: {STRICT_RISK*100}% | Min ROI: {MIN_ROI_ENTRY*100}%")
print("-" * 75)

def get_volatility(encoder_seq):
    # Считаем волатильность по последним 12 свечам
    prices = np.exp(encoder_seq[-12:].cpu().detach().numpy())
    return np.std(prices) / np.mean(prices)

def run_trailing_strategy_verified(name):
    balance = 1000.0
    trades = 0
    wins = 0
    busy_until_idx = -1  # Блокировка счета (чтобы не открывать 10 сделок подряд)

    for i in range(n_samples):
        # 0. Синхронизация времени
        prediction_start_idx = int(index_df.iloc[i]['time_idx'])
        decision_idx = prediction_start_idx - 1

        # Если в сделке - пропускаем
        if decision_idx < busy_until_idx:
            continue

        # --- 1. ВХОД (TITAN SIGNALS) ---
        log_forecast = preds[i, :, 3].cpu().detach().numpy()[-1]
        log_optimist = preds[i, :, 5].cpu().detach().numpy()[-1]
        log_pessimist = preds[i, :, 1].cpu().detach().numpy()[-1]

        log_curr = x["encoder_target"][i][-1].cpu().detach().numpy()
        curr_p = np.exp(log_curr)
        targ_p = np.exp(log_forecast)

        roi = (targ_p - curr_p) / curr_p
        risk_spread = (np.exp(log_optimist) - np.exp(log_pessimist)) / targ_p

        signal = "NONE"
        if risk_spread <= STRICT_RISK:
            if roi > MIN_ROI_ENTRY: signal = "LONG"
            elif roi < -MIN_ROI_ENTRY: signal = "SHORT"

        # --- 2. ВЫХОД (TRAILING STOP) ---
        if signal != "NONE":
            log_future = x["decoder_target"][i].cpu().detach().numpy()
            future_prices = np.exp(log_future)

            # ДИСТАНЦИЯ ТРЕЙЛИНГА
            if "СКАЛЬПЕР" in name:
                TRAIL_PCT = 0.0055  # 0.55%
            elif "ИМПУЛЬС" in name:
                TRAIL_PCT = 0.0075  # 0.75%
            elif "ATR" in name:
                vol = get_volatility(x["encoder_target"][i])
                TRAIL_PCT = max(vol * 1.5, 0.005) # 1.5 ATR или минимум 0.5%

            exit_p = None

            if signal == "LONG":
                # Стартовый стоп
                current_sl = curr_p * (1 - TRAIL_PCT)
                highest_price = curr_p

                for price in future_prices:
                    # Подтяжка стопа (ТОЛЬКО ВВЕРХ)
                    if price > highest_price:
                        highest_price = price
                        new_sl = highest_price * (1 - TRAIL_PCT)
                        if new_sl > current_sl:
                            current_sl = new_sl

                    # Проверка выхода
                    if price <= current_sl:
                        # Выходим по цене стопа (имитация стоп-ордера)
                        exit_p = current_sl
                        break

            elif signal == "SHORT":
                # Стартовый стоп
                current_sl = curr_p * (1 + TRAIL_PCT)
                lowest_price = curr_p

                for price in future_prices:
                    # Подтяжка стопа (ТОЛЬКО ВНИЗ)
                    if price < lowest_price:
                        lowest_price = price
                        new_sl = lowest_price * (1 + TRAIL_PCT)
                        if new_sl < current_sl:
                            current_sl = new_sl

                    # Проверка выхода
                    if price >= current_sl:
                        # Выходим по цене стопа
                        exit_p = current_sl
                        break

            # Если стоп не выбило за 12 часов - выход по последней цене
            if exit_p is None:
                exit_p = future_prices[-1]

            # --- 3. РАСЧЕТ PNL ---
            # Защита от нулевых цен
            if exit_p < 0.0001: continue

            if signal == "LONG":
                raw_pnl = (exit_p - curr_p) / curr_p
            else:
                raw_pnl = (curr_p - exit_p) / curr_p

            net_pnl = raw_pnl - COMMISSION
            balance *= (1 + net_pnl)

            trades += 1
            if net_pnl > 0: wins += 1

            # Блокируем торговлю на время удержания (макс 12 часов)
            busy_until_idx = decision_idx + 12

    # ВЫВОД
    color = "\033[92m" if balance > 1000 else "\033[91m"
    res_pct = (balance - 1000) / 10

    print(f"📌 {name:<15} | Deals: {trades:3} | WR: {wins/trades*100 if trades>0 else 0:.1f}% | {color}${balance:.2f} ({res_pct:+.1f}%)\033[0m")

# ЗАПУСК
print(f"{'STRATEGY':<17} | {'STATS':<20} | {'RESULT'}")
print("-" * 75)

run_trailing_strategy_verified("А. СКАЛЬПЕР")
run_trailing_strategy_verified("Б. ИМПУЛЬС")
run_trailing_strategy_verified("В. ATR (SMART)")

print("-" * 75)

✅ Данные синхронизированы. Доступно семплов: 709
💎 ЗАПУСК TRAILING STOP (С УЧЕТОМ ПРОСКАЛЬЗЫВАНИЯ)
🔒 Risk Limit: 2.8000000000000003% | Min ROI: 0.5499999999999999%
---------------------------------------------------------------------------
STRATEGY          | STATS                | RESULT
---------------------------------------------------------------------------
📌 А. СКАЛЬПЕР     | Deals:  10 | WR: 50.0% | $1000.80 (+0.1%)
📌 Б. ИМПУЛЬС      | Deals:  10 | WR: 60.0% | $1018.27 (+1.8%)
📌 В. ATR (SMART)  | Deals:  10 | WR: 60.0% | $1022.80 (+2.3%)
---------------------------------------------------------------------------
